# **LIBRARIES IMPORT**

In [1]:
import sys
sys.path.append('D:\\Projects\\nlp-projects\\utils')
print(sys.path)

['d:\\Projects\\nlp-projects\\lab-2', 'C:\\Users\\legion\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\legion\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\legion\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\legion\\AppData\\Local\\Programs\\Python\\Python312', 'd:\\Projects\\nlp-projects\\.venv', '', 'd:\\Projects\\nlp-projects\\.venv\\Lib\\site-packages', 'd:\\Projects\\nlp-projects\\.venv\\Lib\\site-packages\\win32', 'd:\\Projects\\nlp-projects\\.venv\\Lib\\site-packages\\win32\\lib', 'd:\\Projects\\nlp-projects\\.venv\\Lib\\site-packages\\Pythonwin', 'D:\\Projects\\nlp-projects\\utils']


In [2]:
import json
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import clean
import stem
import utils
import tagger
from tashaphyne.stemming import ArabicLightStemmer
import qalsadi.lemmatizer
from farasa.pos import FarasaPOSTagger 
from farasa.ner import FarasaNamedEntityRecognizer 
from farasa.stemmer import FarasaStemmer
import bill_generator as bg
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from gensim.models import FastText
from sklearn.manifold import TSNE
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.scripts.glove2word2vec import glove2word2vec

# **Bill generator test**

In [3]:
text = 'I bought two million twenty three thousand and forty nine Samsung smartphones 150 $ each, 4 kg of fresh banana for 1,2 dollar a kilogram, and one Hamburger with 4,5 dollar'

bill = bg.generate_bill(text)
bg.print_bill(bill)

Product             | Quantity | Unit Price | Total Price
---------------------------------------------------------
Samsung smartphones | 2023049  | 150.0      | 303457350.0
kg fresh banana     | 4        | 1.2        | 4.8
Hamburger           | 1        | 4.5        | 4.5


# **DATA**

In [4]:
# If you are on Google Colab:
# with open('/content/tmp/aljazzera.json', 'r', encoding='utf-8') as json_file:
with open('../lab-1/tmp/aljazzera.json', 'r', encoding='utf-8') as json_file:
    articles = json.load(json_file)
    articleExample = articles[0]

articleExample

{'title': 'ترامب يصف المهاجرين بأنهم "حيوانات"',
 'link': '/news/2024/4/3/%d8%aa%d8%b1%d8%a7%d9%85%d8%a8-%d9%8a%d8%b5%d9%81-%d8%a7%d9%84%d9%85%d9%87%d8%a7%d8%ac%d8%b1%d9%8a%d9%86-%d8%a8%d8%a3%d9%86%d9%87%d9%85-%d8%ad%d9%8a%d9%88%d8%a7%d9%86%d8%a7%d8%aa',
 'date': '3/4/2024',
 'content': 'وصف المرشح الجمهوري للرئاسة الأميركية، دونالد ترامب، المهاجرين غير الشرعيين في الولايات المتحدة بأنهم "حيوانات" و"ليسوا بشرا" خلال خطاب ألقاه في ولاية ميشيغان، مما أثار انتقادات واسعة لعودته إلى اللغة المهينة التي اعتمدها في حملاته السابقة.\nوجاءت تصريحات ترامب خلال ظهوره إلى جانب عدد من ضباط إنفاذ القانون، حيث تطرق إلى عدة قضايا جنائية تتعلق بمشتبه بهم موجودين في الولايات المتحدة بشكل غير قانوني، وحذر من أن العنف والفوضى سيدمران البلاد إذا لم يحقق الفوز في الانتخابات المقررة في نوفمبر/تشرين الثاني.\nوفي خطاب آخر في ولاية ويسكونسن، واصل ترامب نبرة التصريحات المثيرة، حيث وصف انتخابات عام 2024 بأنها "المعركة النهائية" للأمة.\nوفيما يتعلق بحادثة لاكين رايلي، طالبة التمريض البالغة من العمر 22 عاما من جورجي

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\legion\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\legion\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\legion\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# **DATA WRANGLING & PROCESSING PIPELINE**
This is the most important part of the project. Here we will be cleaning the data and making it ready for the analysis.
Our pipeline will have the following steps:
1. **Removing Punctuation**
2. **Discretization**
3. **Tokenization**
4. **Normalization**
5. **Removing Stopwords**

The pipeline input will be an article in string format and the output will be a list of words.

Each step of the pipeline will print its output for better understanding.

We will also be adding necessary processing if needed.

### **Removing punctuation, discretization, normalization and paragraphs tokenization**

In [6]:
content = articleExample['content']

content = clean.remove_punctuations(content)
content = clean.remove_diacritics(content)
content = clean.normalize_arabic(content)

contentParagraphs = content.split('\n')

contentParagraphs = list(filter(None, contentParagraphs))
res = []
for ele in contentParagraphs:
    if ele.strip():
        res.append(ele)

contentParagraphs = res
contentParagraphs

['وصف المرشح الجمهوري للرءاسه الاميركيه دونالد ترامب المهاجرين غير الشرعيين في الولايات المتحده بانهم حيوانات وليسوا بشرا خلال خطاب القاه في ولايه ميشيغان مما اثار انتقادات واسعه لعودته الي اللغه المهينه التي اعتمدها في حملاته السابقه',
 'وجاءت تصريحات ترامب خلال ظهوره الي جانب عدد من ضباط انفاذ القانون حيث تطرق الي عده قضايا جناءيه تتعلق بمشتبه بهم موجودين في الولايات المتحده بشكل غير قانوني وحذر من ان العنف والفوضي سيدمران البلاد اذا لم يحقق الفوز في الانتخابات المقرره في نوفمبرتشرين الثاني',
 'وفي خطاب اخر في ولايه ويسكونسن واصل ترامب نبره التصريحات المثيره حيث وصف انتخابات عام 2024 بانها المعركه النهاءيه للامه',
 'وفيما يتعلق بحادثه لاكين رايلي طالبه التمريض البالغه من العمر 22 عاما من جورجيا والتي يزعم انها قتلت علي يد مهاجر غير شرعي من فنزويلا اشار ترامب الي ان بعض المهاجرين ليسوا بشرا',
 'وفي تعليقه قال ترامب يقول الديمقراطيون من فضلكم لا تطلقوا عليهم اسم الحيوانات انهم بشر انا اقول لا انهم ليسوا بشرا انهم حيوانات',
 'وفي خطاباته المثيره للجدل يدعي ترامب في كثير من الاحيان ان ال

### **Sentences tokenization**

In [7]:
for p in contentParagraphs:
    p = sent_tokenize(p)
    print(p)

['وصف المرشح الجمهوري للرءاسه الاميركيه دونالد ترامب المهاجرين غير الشرعيين في الولايات المتحده بانهم حيوانات وليسوا بشرا خلال خطاب القاه في ولايه ميشيغان مما اثار انتقادات واسعه لعودته الي اللغه المهينه التي اعتمدها في حملاته السابقه']
['وجاءت تصريحات ترامب خلال ظهوره الي جانب عدد من ضباط انفاذ القانون حيث تطرق الي عده قضايا جناءيه تتعلق بمشتبه بهم موجودين في الولايات المتحده بشكل غير قانوني وحذر من ان العنف والفوضي سيدمران البلاد اذا لم يحقق الفوز في الانتخابات المقرره في نوفمبرتشرين الثاني']
['وفي خطاب اخر في ولايه ويسكونسن واصل ترامب نبره التصريحات المثيره حيث وصف انتخابات عام 2024 بانها المعركه النهاءيه للامه']
['وفيما يتعلق بحادثه لاكين رايلي طالبه التمريض البالغه من العمر 22 عاما من جورجيا والتي يزعم انها قتلت علي يد مهاجر غير شرعي من فنزويلا اشار ترامب الي ان بعض المهاجرين ليسوا بشرا']
['وفي تعليقه قال ترامب يقول الديمقراطيون من فضلكم لا تطلقوا عليهم اسم الحيوانات انهم بشر انا اقول لا انهم ليسوا بشرا انهم حيوانات']
['وفي خطاباته المثيره للجدل يدعي ترامب في كثير من الاحيان ان ال

### **Removing stopwords and words tokenization**

In [8]:
stop_words = set(stopwords.words('arabic'))
contentWords = []

for p in contentParagraphs:
    words = word_tokenize(p)
    print("original:")
    print(words)
    words = [w for w in words if not w in stop_words]
    print("without stopwords:")
    print(words)
    print("--------------------------------------")
    contentWords.append(words)

original:
['وصف', 'المرشح', 'الجمهوري', 'للرءاسه', 'الاميركيه', 'دونالد', 'ترامب', 'المهاجرين', 'غير', 'الشرعيين', 'في', 'الولايات', 'المتحده', 'بانهم', 'حيوانات', 'وليسوا', 'بشرا', 'خلال', 'خطاب', 'القاه', 'في', 'ولايه', 'ميشيغان', 'مما', 'اثار', 'انتقادات', 'واسعه', 'لعودته', 'الي', 'اللغه', 'المهينه', 'التي', 'اعتمدها', 'في', 'حملاته', 'السابقه']
without stopwords:
['وصف', 'المرشح', 'الجمهوري', 'للرءاسه', 'الاميركيه', 'دونالد', 'ترامب', 'المهاجرين', 'الشرعيين', 'الولايات', 'المتحده', 'بانهم', 'حيوانات', 'وليسوا', 'بشرا', 'خلال', 'خطاب', 'القاه', 'ولايه', 'ميشيغان', 'اثار', 'انتقادات', 'واسعه', 'لعودته', 'الي', 'اللغه', 'المهينه', 'اعتمدها', 'حملاته', 'السابقه']
--------------------------------------
original:
['وجاءت', 'تصريحات', 'ترامب', 'خلال', 'ظهوره', 'الي', 'جانب', 'عدد', 'من', 'ضباط', 'انفاذ', 'القانون', 'حيث', 'تطرق', 'الي', 'عده', 'قضايا', 'جناءيه', 'تتعلق', 'بمشتبه', 'بهم', 'موجودين', 'في', 'الولايات', 'المتحده', 'بشكل', 'غير', 'قانوني', 'وحذر', 'من', 'ان', 'العنف', 'والفوض

# **Stemming and Lemmatization**
## **Stemming - implementation**

In [9]:
contentWordsStemmed = []

for pw in contentWords:
    words = stem.light_stem(pw)
    print(words)
    contentWordsStemmed.append(words)

['وصف', 'مرشح', 'جمهوري', 'رءاسه', 'اميركيه', 'دونالد', 'ترامب', 'مهاجر', 'شرعي', 'ولاي', 'متحده', 'بان', 'حيوان', 'وليس', 'بشرا', 'خلال', 'خطاب', 'قاه', 'ولايه', 'ميشيغ', 'اثار', 'انتقاد', 'واسعه', 'لعودته', 'الي', 'لغه', 'مهينه', 'اعتمد', 'حملاته', 'سابقه']
['وجاءت', 'تصريح', 'ترامب', 'خلال', 'ظهوره', 'الي', 'جانب', 'عدد', 'ضباط', 'انفاذ', 'قان', 'تطرق', 'الي', 'عده', 'قضا', 'جناءيه', 'تتعلق', 'بمشتبه', 'موجود', 'ولاي', 'متحده', 'بشكل', 'قانو', 'وحذر', 'ان', 'عنف', 'فوضي', 'سيدمر', 'بلاد', 'اذا', 'يحقق', 'فوز', 'انتخاب', 'مقرره', 'نوفمبرتشر', 'ثاني']
['وفي', 'خطاب', 'اخر', 'ولايه', 'ويسكونسن', 'واصل', 'ترامب', 'نبره', 'تصريح', 'مثيره', 'وصف', 'انتخاب', 'عام', '2024', 'بان', 'معركه', 'نهاءيه', 'امه']
['وفي', 'يتعلق', 'بحادثه', 'لاك', 'رايلي', 'طالبه', 'تمريض', 'بالغه', 'عمر', '22', 'عاما', 'جورج', 'والتي', 'يزعم', 'انها', 'قتلت', 'علي', 'يد', 'مهاجر', 'شرعي', 'فنزويلا', 'اشار', 'ترامب', 'الي', 'ان', 'مهاجر', 'بشرا']
['وفي', 'تعليقه', 'قال', 'ترامب', 'يقول', 'ديمقراطي', 'فضل', 'تطلق', 

## **Lemmatization**

In [10]:
lemmatizer = WordNetLemmatizer()
contentWordsLemmatized = []

for pw in contentWords:
    words = [lemmatizer.lemmatize(w) for w in pw]
    print(words)
    contentWordsLemmatized.append(words)

['وصف', 'المرشح', 'الجمهوري', 'للرءاسه', 'الاميركيه', 'دونالد', 'ترامب', 'المهاجرين', 'الشرعيين', 'الولايات', 'المتحده', 'بانهم', 'حيوانات', 'وليسوا', 'بشرا', 'خلال', 'خطاب', 'القاه', 'ولايه', 'ميشيغان', 'اثار', 'انتقادات', 'واسعه', 'لعودته', 'الي', 'اللغه', 'المهينه', 'اعتمدها', 'حملاته', 'السابقه']
['وجاءت', 'تصريحات', 'ترامب', 'خلال', 'ظهوره', 'الي', 'جانب', 'عدد', 'ضباط', 'انفاذ', 'القانون', 'تطرق', 'الي', 'عده', 'قضايا', 'جناءيه', 'تتعلق', 'بمشتبه', 'موجودين', 'الولايات', 'المتحده', 'بشكل', 'قانوني', 'وحذر', 'ان', 'العنف', 'والفوضي', 'سيدمران', 'البلاد', 'اذا', 'يحقق', 'الفوز', 'الانتخابات', 'المقرره', 'نوفمبرتشرين', 'الثاني']
['وفي', 'خطاب', 'اخر', 'ولايه', 'ويسكونسن', 'واصل', 'ترامب', 'نبره', 'التصريحات', 'المثيره', 'وصف', 'انتخابات', 'عام', '2024', 'بانها', 'المعركه', 'النهاءيه', 'للامه']
['وفيما', 'يتعلق', 'بحادثه', 'لاكين', 'رايلي', 'طالبه', 'التمريض', 'البالغه', 'العمر', '22', 'عاما', 'جورجيا', 'والتي', 'يزعم', 'انها', 'قتلت', 'علي', 'يد', 'مهاجر', 'شرعي', 'فنزويلا', 'اشار',

## **Stemming with araby**

In [11]:
ArListem = ArabicLightStemmer()

contentWordsStemmed = []

for pw in contentWords:
    words = [ArListem.light_stem(w) for w in pw]
    print(words)
    contentWordsStemmed.append(words)

['صف', 'مرشح', 'جمهور', 'رءاسه', 'اميركيه', 'دونالد', 'ترامب', 'مهاجر', 'شرعي', 'ولا', 'متحده', 'ان', 'حيوان', 'ليسوا', 'شرا', 'خلال', 'خطاب', 'قاه', 'اي', 'ميشيغ', 'ثار', 'انتقاد', 'سع', 'عود', 'لي', 'لغه', 'مهينه', 'عتمد', 'حمل', 'سابقه']
['جاء', 'تصريح', 'ترامب', 'خلال', 'ظهور', 'لي', 'جانب', 'عدد', 'ضباط', 'نفاذ', 'قان', 'طرق', 'لي', 'عد', 'قضاي', 'جناء', 'تعلق', 'مشتب', 'موجود', 'ولا', 'متحده', 'شكل', 'قان', 'حذر', 'ان', 'عنف', 'فوض', 'دمر', 'بلاد', 'ذا', 'حقق', 'فوز', 'انتخاب', 'مقرره', 'نوفمبرتشرين', 'ثان']
['في', 'خطاب', 'خر', 'اي', 'يسكونسن', 'صل', 'ترامب', 'بر', 'تصريح', 'مثيره', 'صف', 'انتخاب', 'عام', '2024', 'ان', 'معركه', 'نهاءيه', 'امه']
['فيما', 'تعلق', 'حادث', 'اك', 'رايل', 'طالب', 'تمريض', 'بالغه', 'عمر', '22', 'عام', 'جورجي', 'التي', 'زعم', 'نه', 'قتل', 'على', 'يد', 'مهاجر', 'شرع', 'نزويل', 'شار', 'ترامب', 'لي', 'ان', 'مهاجر', 'شرا']
['في', 'عليق', 'قال', 'ترامب', 'قول', 'الديمقراطيون', 'ضل', 'طلق', 'على', 'سم', 'حيوان', 'نهم', 'شر', 'نا', 'قول', 'نهم', 'شرا', 'نهم', 

## **Lemmatization with qalsadi**

In [12]:
lemmer = qalsadi.lemmatizer.Lemmatizer()

contentWordsLemmatized = []

for pw in contentWords:
    words = [lemmer.lemmatize(w) for w in pw]
    print(words)
    contentWordsLemmatized.append(words)

['صف', 'مرشح', 'جمهور', 'للرءاسه', 'الاميركيه', 'دونالد', 'ترامب', 'مهاجر', 'شرعي', 'ولاية', 'المتحده', 'بان', 'حيوان', 'ليسوا', 'بشر', 'خلال', 'خطاب', 'القاه', 'ولايه', 'ميشيغان', 'اثار', 'انتقاد', 'واسع', 'عود', 'ال', 'اللغه', 'المهينه', 'اعتمد', 'حمل', 'السابقه']
['جاء', 'تصريح', 'ترامب', 'خلال', 'ظهور', 'ال', 'جانب', 'عدد', 'ضباط', 'انفاذ', 'قانون', 'تطرق', 'ال', 'عد', 'قضايا', 'جناءيه', 'تعلق', 'مشتبه', 'موجود', 'ولاية', 'المتحده', 'شكل', 'قانون', 'حذر', 'ان', 'عنف', 'والفوضي', 'دمر', 'بلاد', 'اذا', 'أحق', 'فوز', 'انتخاب', 'المقرره', 'نوفمبرتشرين', 'ثاني']
['في', 'خطاب', 'اخر', 'ولايه', 'ويسكونسن', 'واصل', 'ترامب', 'نبر', 'تصريح', 'المثيره', 'صف', 'انتخاب', 'عام', '2024', 'بان', 'المعركه', 'النهاءيه', 'لام']
['فيما', 'تعلق', 'حادث', 'لاك', 'رايلي', 'طالب', 'تمريض', 'البالغه', 'عمر', '22', 'عام', 'جورجي', 'التي', 'زعم', 'انها', 'قتل', 'علي', 'يد', 'مهاجر', 'شرع', 'فنزويلا', 'اشار', 'ترامب', 'ال', 'ان', 'مهاجر', 'بشر']
['في', 'تعليق', 'قال', 'ترامب', 'أقول', 'ديمقراطي', 'فضل', 'طلق'

## **Stemming with farasa**
Farasa is a fast and accurate modern Arabic NLP toolkit, developed by [Arabic Language Technologies Group](https://alt.qcri.org/) at [Qatar Computing Research Institute (QCRI)](https://www.hbku.edu.qa/en/qcri). The python package we are using is a wrapper around the original Java package since the original package is not developed in python. 

In [13]:
farasaStemmer = FarasaStemmer()

#### **Stemming tokens with farasa**

In [ ]:
# NOT RECOMMENDED DUE TO THE TIME IT TAKES AND PERFORMANCE
contentWordsStemmed = []

for pw in contentWords:
    words = [farasaStemmer.stem(w) for w in pw]
    print(words)
    contentWordsStemmed.append(words)

The Farasa library is so HEAVY computationally, so using it on each token is not a good idea. Instead, we can use it on the whole text. This way, we can reduce the number of calls to the library. Then we will apply the same preprocessing steps as we done first.

That's why we will stem a whole text, POS tag it, and then apply the same preprocessing steps as we did before.

In [14]:
stemmedTextFarasa = farasaStemmer.stem(content)
print("Stemmed Text:\n", stemmedTextFarasa)

Stemmed Text:
 وصف مرشح جمهوري رءاسه اميركيه دونالد ترامب مهاجر غير شرعي في ولاية المتحده أن حيوان ليس بشر خلال خطاب ألقى في لا ميشيغان من اثار انتقاد واسع عودة الي لغه مهينه الذي اعتمد في حملة سابق
جاء تصريح ترامب خلال ظهور الي جانب عدد من ضابط إنفاذ قانون حيث تطرق الي عد قضية جناءيه تعلق مشتبه ب موجود في ولاية المتحده شكل غير قانوني حذر من ان عنف فوضي دمر بلد اذ لم حقق فوز في انتخاب مقرره في نوفمبرتشرين ثاني
في خطاب آخر في لا ويسكونسن واصل ترامب نبر تصريح مثيره حيث وصف انتخاب عام 2024 أن معركه نهاءيه لم
في تعلق حادث لاك رايلي طالب تمريض بالغ من عمر 22 عام من جورجيا الذي زعم ان قتل علي يد مهاجر غير شرعي من فنزويلا أشار ترامب الي ان بعض مهاجر ليس بشر
في تعليق قال ترامب قال ديموقراطي من فضل لا أطلق على اسم حيوان ان بشر أنا أقول لا ان ليس بشر ان حيوان


في خطاب مثيره جدل ادعى ترامب في كثير من حين ان مهاجر الذي عبر حد مع مكسيك شكل غير قانوني قد فر من سجن ملاجء في بلد اصليه من زاد من جراءم عنف في ولاية المتحده
اتهم منافس في انتخاب رءاسيه رءيس جو بايدن تحفيز جمهوري في كونجرس علي عدم إقرار ت

# **POS TAGGING & NER**
## **POS tagging with Farasa**
#### **POS tagging the uncleaned text**

In [18]:
pos_tagger = FarasaPOSTagger()
POSTaggedText = pos_tagger.tag(stemmedTextFarasa)

print("sample POS Tagged\n", POSTaggedText)

sample POS Tagged
 S/S وصف/V مرشح/NOUN-MS جمهوري/ADJ-MS رءاس/NOUN-MS +ه/PRON اميركي/NOUN-MS +ه/PRON دونالد/NOUN-MS ترامب/NOUN-MS مهاجر/NOUN-MS غير/NOUN-MS شرعي/ADJ-MS في/PREP ولاي +ة/NOUN+NSUFF-FS المتحده/NOUN-MS أن/PART حيوان/NOUN-MS ليس/V بشر/NOUN-MS خلال/NOUN-MS خطاب/NOUN-MS ألقى/V في/PREP لا/PART ميشيغان/NOUN-MS من/PREP اثار/NOUN-FS انتقاد/NOUN-MS واسع/ADJ-MS عود +ة/NOUN+NSUFF-FS الي/FOREIGN لغ/NOUN-MS +ه/PRON مهين/ADJ-MS +ه/PRON الذي/PART اعتمد/V في/PREP حمل +ة/NOUN+NSUFF-FS سابق/ADJ-MS E/E 
S/S جاء/V تصريح/NOUN-MS ترامب/NOUN-MS خلال/NOUN-MS ظهور/NOUN-MS الي/FOREIGN جانب/NOUN-MS عدد/NOUN-MS من/PREP ضابط/NOUN-MS إنفاذ/NOUN-MS قانون/NOUN-MS حيث/PART تطرق/V الي/FOREIGN عد/NOUN-MS قضي +ة/NOUN+NSUFF-FS جناءيه/NOUN-MS تعلق/V مشتبه/NOUN-MS ب/PREP موجود/ADJ-MS في/PREP ولاي +ة/NOUN+NSUFF-FS المتحده/NOUN-MS شكل/NOUN-MS غير/NOUN-MS قانوني/ADJ-MS حذر/ADJ-MS من/PREP ان/NOUN-MS عنف/NOUN-MS فوضي/ADJ-MS دمر/V بلد/NOUN-MS إذ/PART لم/PART حقق/V فوز/NOUN-MS في/PREP انتخاب/NOUN-MS مقرر/ADJ-MS +ه/PRON

In [19]:
POSTaggedParagraphs = POSTaggedText.split('\n')

POSTaggedParagraphs = list(filter(None, POSTaggedParagraphs))
res = []
for ele in POSTaggedParagraphs:
    if ele.strip():
        res.append(ele)

POSTaggedParagraphs = res
POSTaggedParagraphs

['S/S وصف/V مرشح/NOUN-MS جمهوري/ADJ-MS رءاس/NOUN-MS +ه/PRON اميركي/NOUN-MS +ه/PRON دونالد/NOUN-MS ترامب/NOUN-MS مهاجر/NOUN-MS غير/NOUN-MS شرعي/ADJ-MS في/PREP ولاي +ة/NOUN+NSUFF-FS المتحده/NOUN-MS أن/PART حيوان/NOUN-MS ليس/V بشر/NOUN-MS خلال/NOUN-MS خطاب/NOUN-MS ألقى/V في/PREP لا/PART ميشيغان/NOUN-MS من/PREP اثار/NOUN-FS انتقاد/NOUN-MS واسع/ADJ-MS عود +ة/NOUN+NSUFF-FS الي/FOREIGN لغ/NOUN-MS +ه/PRON مهين/ADJ-MS +ه/PRON الذي/PART اعتمد/V في/PREP حمل +ة/NOUN+NSUFF-FS سابق/ADJ-MS E/E ',
 'S/S جاء/V تصريح/NOUN-MS ترامب/NOUN-MS خلال/NOUN-MS ظهور/NOUN-MS الي/FOREIGN جانب/NOUN-MS عدد/NOUN-MS من/PREP ضابط/NOUN-MS إنفاذ/NOUN-MS قانون/NOUN-MS حيث/PART تطرق/V الي/FOREIGN عد/NOUN-MS قضي +ة/NOUN+NSUFF-FS جناءيه/NOUN-MS تعلق/V مشتبه/NOUN-MS ب/PREP موجود/ADJ-MS في/PREP ولاي +ة/NOUN+NSUFF-FS المتحده/NOUN-MS شكل/NOUN-MS غير/NOUN-MS قانوني/ADJ-MS حذر/ADJ-MS من/PREP ان/NOUN-MS عنف/NOUN-MS فوضي/ADJ-MS دمر/V بلد/NOUN-MS إذ/PART لم/PART حقق/V فوز/NOUN-MS في/PREP انتخاب/NOUN-MS مقرر/ADJ-MS +ه/PRON في/PREP نوفم

In [20]:
POSDict = utils.tagsDict(POSTaggedParagraphs, 'pos')

for w in POSDict:
    print(w)

{'S': 'S', 'وصف': 'V', 'مرشح': 'NOUN-MS', 'جمهوري': 'ADJ-MS', 'رءاس': 'NOUN-MS', '+ه': 'PRON', 'اميركي': 'NOUN-MS', 'دونالد': 'NOUN-MS', 'ترامب': 'NOUN-MS', 'مهاجر': 'NOUN-MS', 'غير': 'NOUN-MS', 'شرعي': 'ADJ-MS', 'في': 'PREP', 'ولاي+ة': 'NOUN+NSUFF-FS', 'المتحده': 'NOUN-MS', 'أن': 'PART', 'حيوان': 'NOUN-MS', 'ليس': 'V', 'بشر': 'NOUN-MS', 'خلال': 'NOUN-MS', 'خطاب': 'NOUN-MS', 'ألقى': 'V', 'لا': 'PART', 'ميشيغان': 'NOUN-MS', 'من': 'PREP', 'اثار': 'NOUN-FS', 'انتقاد': 'NOUN-MS', 'واسع': 'ADJ-MS', 'عود+ة': 'NOUN+NSUFF-FS', 'الي': 'FOREIGN', 'لغ': 'NOUN-MS', 'مهين': 'ADJ-MS', 'الذي': 'PART', 'اعتمد': 'V', 'حمل+ة': 'NOUN+NSUFF-FS', 'سابق': 'ADJ-MS', 'E': 'E'}
{'S': 'S', 'جاء': 'V', 'تصريح': 'NOUN-MS', 'ترامب': 'NOUN-MS', 'خلال': 'NOUN-MS', 'ظهور': 'NOUN-MS', 'الي': 'FOREIGN', 'جانب': 'NOUN-MS', 'عدد': 'NOUN-MS', 'من': 'PREP', 'ضابط': 'NOUN-MS', 'إنفاذ': 'NOUN-MS', 'قانون': 'NOUN-MS', 'حيث': 'PART', 'تطرق': 'V', 'عد': 'NOUN-MS', 'قضي+ة': 'NOUN+NSUFF-FS', 'جناءيه': 'NOUN-MS', 'تعلق': 'V', 'مشت

#### **POS tagging the stemmed text**

In [21]:
POSTaggedParagraphs = []

StemmedContentParagraphs = content.split('\n')
StemmedContentParagraphs = list(filter(None, StemmedContentParagraphs))

res = []
for ele in StemmedContentParagraphs:
    if ele.strip():
        res.append(ele)

StemmedContentParagraphs = res

for p in StemmedContentParagraphs:
        p = sent_tokenize(p)

for p in StemmedContentParagraphs:
    POSTaggedParagraphs.append(pos_tagger.tag(p))
    
POSTaggedParagraphs

['S/S وصف/V ال+ مرشح/DET+NOUN-MS ال+ جمهوري/DET+ADJ-MS ل+/PREP ال+ رءاسه/DET+NOUN-MS ال+ اميركيه/DET+ADJ-MS دونالد/NOUN-MS ترامب/NOUN-MS ال+ مهاجر +ين/DET+NOUN+NSUFF-MP غير/PART ال+ شرعي +ين/DET+ADJ+NSUFF-FD في/PREP ال+ ولاي +ات/DET+NOUN+NSUFF-FP المتحده/NOUN-MS ب+/PREP أن/PART +هم/PRON حيوان +ات/NOUN+NSUFF-FP و+/CONJ ليس +وا/V+PRON بشر/NOUN-MS +ا/CASE خلال/NOUN-MS خطاب/NOUN-MS ألقا/V +ه/PRON في/PREP و+/CONJ لا/PART +ي/PRON +ه/PRON ميشيغان/NOUN-MS مما/PREP+PART اثار/V انتقاد +ات/NOUN+NSUFF-MP واسع/ADJ-MS +ه/PRON ل+/PREP عود +ت/NOUN+NSUFF-FD +ه/PRON الي/FOREIGN ال+ لغه/DET+NOUN-MS ال+ مهينه/DET+ADJ-MS التي/PART اعتمد/V +ها/PRON في/PREP حمل +ات/NOUN+NSUFF-FP +ه/PRON ال+ سابقه/DET+NOUN-MS E/E',
 'S/S و+/CONJ جاء +ت/V+PRON تصريح +ات/NOUN+NSUFF-FP ترامب/NOUN-MS خلال/NOUN-MS ظهور/NOUN-MS +ه/PRON الي/FOREIGN جانب/NOUN-MS عدد/NOUN-MS من/PREP ضباط/NOUN-MP إنفاذ/NOUN-MS ال+ قانون/DET+NOUN-MS حيث/PART تطرق/V الي/FOREIGN عد/V +ه/PRON قضايا/NOUN-FP جناءيه/NOUN-MS تتعلق/V ب+/PREP مشتبه/NOUN-MS ب+/PR

In [22]:
POSDict = utils.tagsDict(POSTaggedParagraphs, 'pos')

for w in POSDict:
    print(w)

{'S': 'S', 'وصف': 'V', 'ال+مرشح': 'DET+NOUN-MS', 'ال+جمهوري': 'DET+ADJ-MS', 'ل+': 'PREP', 'ال+رءاسه': 'DET+NOUN-MS', 'ال+اميركيه': 'DET+ADJ-MS', 'دونالد': 'NOUN-MS', 'ترامب': 'NOUN-MS', 'ال+مهاجر+ين': 'DET+NOUN+NSUFF-MP', 'غير': 'PART', 'ال+شرعي+ين': 'DET+ADJ+NSUFF-FD', 'في': 'PREP', 'ال+ولاي+ات': 'DET+NOUN+NSUFF-FP', 'المتحده': 'NOUN-MS', 'ب+': 'PREP', 'أن': 'PART', '+هم': 'PRON', 'حيوان+ات': 'NOUN+NSUFF-FP', 'و+': 'CONJ', 'ليس+وا': 'V+PRON', 'بشر': 'NOUN-MS', '+ا': 'CASE', 'خلال': 'NOUN-MS', 'خطاب': 'NOUN-MS', 'ألقا': 'V', '+ه': 'PRON', 'لا': 'PART', '+ي': 'PRON', 'ميشيغان': 'NOUN-MS', 'مما': 'PREP+PART', 'اثار': 'V', 'انتقاد+ات': 'NOUN+NSUFF-MP', 'واسع': 'ADJ-MS', 'عود+ت': 'NOUN+NSUFF-FD', 'الي': 'FOREIGN', 'ال+لغه': 'DET+NOUN-MS', 'ال+مهينه': 'DET+ADJ-MS', 'التي': 'PART', 'اعتمد': 'V', '+ها': 'PRON', 'حمل+ات': 'NOUN+NSUFF-FP', 'ال+سابقه': 'DET+NOUN-MS', 'E': 'E'}
{'S': 'S', 'و+': 'CONJ', 'جاء+ت': 'V+PRON', 'تصريح+ات': 'NOUN+NSUFF-FP', 'ترامب': 'NOUN-MS', 'خلال': 'NOUN-MS', 'ظهور': 

#### **POS tagging the lemmatized text**

In [23]:
POSTaggedParagraphs = []

TEXT = [' '.join(p) for p in contentWordsLemmatized]

for p in TEXT:
    POSTaggedParagraphs.append(pos_tagger.tag(p))
    
POSTaggedParagraphs

['S/S صف/NOUN-MS مرشح/ADJ-MS جمهور/NOUN-MS ل+/PREP ال+ رءاسه/DET+NOUN-MS ال+ اميركيه/DET+ADJ-MS دونالد/NOUN-MS ترامب/NOUN-MS مهاجر/NOUN-MS شرعي/ADJ-MS ولاي +ة/NOUN+NSUFF-FS المتحده/NOUN-MS بان/NOUN-MP حيوان/NOUN-MS ليس +وا/V+PRON بشر/NOUN-MS خلال/NOUN-MS خطاب/NOUN-MS ألقا/V +ه/PRON و+/CONJ لا/PART +ي/PRON +ه/PRON ميشيغان/NOUN-MS اثار/NOUN-FS انتقاد/NOUN-MS واسع/ADJ-MS عود/NOUN-MS ال/DET ال+ لغه/DET+ADJ-MS ال+ مهينه/DET+ADJ-MS اعتمد/V حمل/NOUN-MS ال+ سابقه/DET+NOUN-MS E/E',
 'S/S جاء/V تصريح/NOUN-MS ترامب/NOUN-MS خلال/NOUN-MS ظهور/NOUN-MS ال/DET جانب/NOUN-MS عدد/NOUN-MS ضباط/NOUN-MP إنفاذ/NOUN-MS قانون/NOUN-MS تطرق/V ال/DET عد/NOUN-MS قضايا/NOUN-FP جناءيه/NOUN-MS تعلق/V مشتبه/NOUN-MS موجود/ADJ-MS ولاي +ة/NOUN+NSUFF-FS المتحده/NOUN-MS شكل/NOUN-MS قانون/NOUN-MS حذر/V ان/NOUN-MS عنف/NOUN-MS و+/CONJ ال+ فوضي/DET+NOUN-MS دمر/V بلاد/NOUN-FP اذ/NOUN-MS +ا/CASE أحق/ADJ-MS فوز/NOUN-MS انتخاب/NOUN-MS ال+ مقرره/DET+NOUN-MS نوفمبرتشرين/NOUN-MS ثاني/ADJ-MS E/E',
 'S/S في/PREP خطاب/NOUN-MS آخر/ADJ-MS

In [24]:
POSDict = utils.tagsDict(POSTaggedParagraphs, 'pos')

for w in POSDict:
    print(w)

{'S': 'S', 'صف': 'NOUN-MS', 'مرشح': 'ADJ-MS', 'جمهور': 'NOUN-MS', 'ل+': 'PREP', 'ال+رءاسه': 'DET+NOUN-MS', 'ال+اميركيه': 'DET+ADJ-MS', 'دونالد': 'NOUN-MS', 'ترامب': 'NOUN-MS', 'مهاجر': 'NOUN-MS', 'شرعي': 'ADJ-MS', 'ولاي+ة': 'NOUN+NSUFF-FS', 'المتحده': 'NOUN-MS', 'بان': 'NOUN-MP', 'حيوان': 'NOUN-MS', 'ليس+وا': 'V+PRON', 'بشر': 'NOUN-MS', 'خلال': 'NOUN-MS', 'خطاب': 'NOUN-MS', 'ألقا': 'V', '+ه': 'PRON', 'و+': 'CONJ', 'لا': 'PART', '+ي': 'PRON', 'ميشيغان': 'NOUN-MS', 'اثار': 'NOUN-FS', 'انتقاد': 'NOUN-MS', 'واسع': 'ADJ-MS', 'عود': 'NOUN-MS', 'ال': 'DET', 'ال+لغه': 'DET+ADJ-MS', 'ال+مهينه': 'DET+ADJ-MS', 'اعتمد': 'V', 'حمل': 'NOUN-MS', 'ال+سابقه': 'DET+NOUN-MS', 'E': 'E'}
{'S': 'S', 'جاء': 'V', 'تصريح': 'NOUN-MS', 'ترامب': 'NOUN-MS', 'خلال': 'NOUN-MS', 'ظهور': 'NOUN-MS', 'ال': 'DET', 'جانب': 'NOUN-MS', 'عدد': 'NOUN-MS', 'ضباط': 'NOUN-MP', 'إنفاذ': 'NOUN-MS', 'قانون': 'NOUN-MS', 'تطرق': 'V', 'عد': 'NOUN-MS', 'قضايا': 'NOUN-FP', 'جناءيه': 'NOUN-MS', 'تعلق': 'V', 'مشتبه': 'NOUN-MS', 'موجود': '

In [25]:
MyOwnTaggedParagraphs = []

for p in contentWordsLemmatized:
    MyOwnTaggedParagraphs.append(tagger.dumbPOSTagger(' '.join(p)))

for p in MyOwnTaggedParagraphs:
    print(p)

{'صف': 'NOUN', 'مرشح': 'NOUN', 'جمهور': 'NOUN', 'للرءاسه': 'NOUN', 'الاميركيه': 'NOUN', 'دونالد': 'NOUN', 'ترامب': 'NOUN', 'مهاجر': 'NOUN', 'شرعي': 'NOUN', 'ولاية': 'NOUN', 'المتحده': 'NOUN', 'بان': 'NOUN', 'حيوان': 'NOUN', 'ليسوا': 'VERB', 'بشر': 'NOUN', 'خلال': 'NOUN', 'خطاب': 'NOUN', 'القاه': 'NOUN', 'ولايه': 'NOUN', 'ميشيغان': 'NOUN', 'اثار': 'NOUN', 'انتقاد': 'NOUN', 'واسع': 'NOUN', 'عود': 'NOUN', 'ال': 'NOUN', 'اللغه': 'NOUN', 'المهينه': 'NOUN', 'اعتمد': 'VERB', 'حمل': 'NOUN', 'السابقه': 'NOUN'}
{'جاء': 'NOUN', 'تصريح': 'NOUN', 'ترامب': 'NOUN', 'خلال': 'NOUN', 'ظهور': 'ADJ', 'ال': 'NOUN', 'جانب': 'NOUN', 'عدد': 'NOUN', 'ضباط': 'NOUN', 'انفاذ': 'VERB', 'قانون': 'NOUN', 'تطرق': 'NOUN', 'عد': 'NOUN', 'قضايا': 'NOUN', 'جناءيه': 'NOUN', 'تعلق': 'NOUN', 'مشتبه': 'NOUN', 'موجود': 'NOUN', 'ولاية': 'NOUN', 'المتحده': 'NOUN', 'شكل': 'NOUN', 'حذر': 'NOUN', 'ان': 'NOUN', 'عنف': 'NOUN', 'والفوضي': 'NOUN', 'دمر': 'NOUN', 'بلاد': 'NOUN', 'اذا': 'NOUN', 'أحق': 'NOUN', 'فوز': 'NOUN', 'انتخاب': 'N

## **Named Entity Recognition (NER) with Farasa**

In [26]:
named_entity_recognizer = FarasaNamedEntityRecognizer()

NamedEntityRecognized = named_entity_recognizer.recognize(stemmedTextFarasa)
print("sample named entity recognized:\n", NamedEntityRecognized)

sample named entity recognized:
 وصف/O مرشح/O جمهوري/O رءاسه/O اميركيه/O دونالد/B-PERS ترامب/I-PERS مهاجر/O غير/O شرعي/O في/O ولاية/O المتحده/B-PERS أن/O حيوان/O ليس/O بشر/O خلال/O خطاب/O ألقى/O في/O لا/O ميشيغان/B-LOC من/O اثار/O انتقاد/O واسع/O عودة/O الي/O لغه/O مهينه/O الذي/O اعتمد/O في/O حملة/O سابق/O
جاء/O تصريح/O ترامب/O خلال/O ظهور/O الي/O جانب/O عدد/O من/O ضابط/O إنفاذ/O قانون/O حيث/O تطرق/O الي/O عد/O قضية/O جناءيه/O تعلق/O مشتبه/O ب/O موجود/O في/O ولاية/O المتحده/O شكل/O غير/O قانوني/O حذر/O من/O ان/O عنف/O فوضي/O دمر/O بلد/O اذ/O لم/O حقق/O فوز/O في/O انتخاب/O مقرره/O في/O نوفمبرتشرين/O ثاني/O
في/O خطاب/O آخر/O في/O لا/O ويسكونسن/O واصل/O ترامب/O نبر/B-PERS تصريح/O مثيره/O حيث/O وصف/O انتخاب/O عام/O 2024/O أن/O معركه/O نهاءيه/O لم/O
في/O تعلق/O حادث/O لاك/O رايلي/B-PERS طالب/O تمريض/O بالغ/O من/O عمر/O 22/O عام/O من/O جورجيا/B-LOC الذي/O زعم/O ان/O قتل/O علي/O يد/O مهاجر/O غير/O شرعي/O من/O فنزويلا/B-LOC أشار/O ترامب/O الي/O ان/O بعض/O مهاجر/O ليس/O بشر/O
في/O تعليق/O قال/O

In [27]:
NERParagraphs = NamedEntityRecognized.split('\n')

NERParagraphs = list(filter(None, NERParagraphs))
res = []
for ele in NERParagraphs:
    if ele.strip():
        res.append(ele)

NERParagraphs = res
NERParagraphs

['وصف/O مرشح/O جمهوري/O رءاسه/O اميركيه/O دونالد/B-PERS ترامب/I-PERS مهاجر/O غير/O شرعي/O في/O ولاية/O المتحده/B-PERS أن/O حيوان/O ليس/O بشر/O خلال/O خطاب/O ألقى/O في/O لا/O ميشيغان/B-LOC من/O اثار/O انتقاد/O واسع/O عودة/O الي/O لغه/O مهينه/O الذي/O اعتمد/O في/O حملة/O سابق/O',
 'جاء/O تصريح/O ترامب/O خلال/O ظهور/O الي/O جانب/O عدد/O من/O ضابط/O إنفاذ/O قانون/O حيث/O تطرق/O الي/O عد/O قضية/O جناءيه/O تعلق/O مشتبه/O ب/O موجود/O في/O ولاية/O المتحده/O شكل/O غير/O قانوني/O حذر/O من/O ان/O عنف/O فوضي/O دمر/O بلد/O اذ/O لم/O حقق/O فوز/O في/O انتخاب/O مقرره/O في/O نوفمبرتشرين/O ثاني/O',
 'في/O خطاب/O آخر/O في/O لا/O ويسكونسن/O واصل/O ترامب/O نبر/B-PERS تصريح/O مثيره/O حيث/O وصف/O انتخاب/O عام/O 2024/O أن/O معركه/O نهاءيه/O لم/O',
 'في/O تعلق/O حادث/O لاك/O رايلي/B-PERS طالب/O تمريض/O بالغ/O من/O عمر/O 22/O عام/O من/O جورجيا/B-LOC الذي/O زعم/O ان/O قتل/O علي/O يد/O مهاجر/O غير/O شرعي/O من/O فنزويلا/B-LOC أشار/O ترامب/O الي/O ان/O بعض/O مهاجر/O ليس/O بشر/O',
 'في/O تعليق/O قال/O ترامب/O قال/O 

In [28]:
NamedEntityRecognizedWords = utils.tagsDict(NERParagraphs, 'ner')

for w in NamedEntityRecognizedWords:
    print(w)

{'وصف': 'O', 'مرشح': 'O', 'جمهوري': 'O', 'رءاسه': 'O', 'اميركيه': 'O', 'دونالد': 'B-PERS', 'ترامب': 'I-PERS', 'مهاجر': 'O', 'غير': 'O', 'شرعي': 'O', 'في': 'O', 'ولاية': 'O', 'المتحده': 'B-PERS', 'أن': 'O', 'حيوان': 'O', 'ليس': 'O', 'بشر': 'O', 'خلال': 'O', 'خطاب': 'O', 'ألقى': 'O', 'لا': 'O', 'ميشيغان': 'B-LOC', 'من': 'O', 'اثار': 'O', 'انتقاد': 'O', 'واسع': 'O', 'عودة': 'O', 'الي': 'O', 'لغه': 'O', 'مهينه': 'O', 'الذي': 'O', 'اعتمد': 'O', 'حملة': 'O', 'سابق': 'O'}
{'جاء': 'O', 'تصريح': 'O', 'ترامب': 'O', 'خلال': 'O', 'ظهور': 'O', 'الي': 'O', 'جانب': 'O', 'عدد': 'O', 'من': 'O', 'ضابط': 'O', 'إنفاذ': 'O', 'قانون': 'O', 'حيث': 'O', 'تطرق': 'O', 'عد': 'O', 'قضية': 'O', 'جناءيه': 'O', 'تعلق': 'O', 'مشتبه': 'O', 'ب': 'O', 'موجود': 'O', 'في': 'O', 'ولاية': 'O', 'المتحده': 'O', 'شكل': 'O', 'غير': 'O', 'قانوني': 'O', 'حذر': 'O', 'ان': 'O', 'عنف': 'O', 'فوضي': 'O', 'دمر': 'O', 'بلد': 'O', 'اذ': 'O', 'لم': 'O', 'حقق': 'O', 'فوز': 'O', 'انتخاب': 'O', 'مقرره': 'O', 'نوفمبرتشرين': 'O', 'ثاني': 'O'}

In [29]:
NamedEntityRecognizedWords = utils.tagsDict(NERParagraphs, 'ner', True)

for w in NamedEntityRecognizedWords:
    print(w)

{'دونالد': 'B-PERS', 'ترامب': 'I-PERS', 'المتحده': 'B-PERS', 'ميشيغان': 'B-LOC'}
{}
{'نبر': 'B-PERS'}
{'رايلي': 'B-PERS', 'جورجيا': 'B-LOC', 'فنزويلا': 'B-LOC'}
{}
{'المتحده': 'B-LOC'}
{'جو': 'B-PERS', 'بايدن': 'I-PERS', 'كونجرس': 'B-ORG'}
{'مايكل': 'B-PERS', 'تايلر': 'I-PERS', 'بايدن': 'B-PERS', 'المتحده': 'B-LOC'}
{'رويترزابسوس': 'B-PERS', 'نهاي': 'I-PERS', 'فبرايرشباط': 'B-PERS', 'ماضي': 'I-PERS', 'المتحده': 'B-LOC'}
{}
{'ميشيغان': 'B-PERS', 'ويسكنسن': 'I-PERS', 'بايدن': 'B-PERS', 'ترامب': 'I-PERS'}
{'بايدن': 'B-PERS', 'ويسكنسن': 'B-LOC', 'ميشيغان': 'B-LOC'}


# **One Hot Encoding**

In [9]:
to_be_encoded = np.array(contentWords[0]).reshape(-1, 1)

enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(to_be_encoded)

for p in enc.fit_transform(to_be_encoded).toarray():
    print(p)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

# **Bag of Words**

In [10]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(contentParagraphs)
df_bow_sklearn = pd.DataFrame(X.toarray(),
columns=vectorizer.get_feature_names_out())

df_bow_sklearn

,2020,2024,22,38,اتصالات,اثار,اجراءات,اجرته,اخر,اذا,...,يحقق,يخوض,يد,يدعي,يرون,يزعم,يزيد,يعبرون,يقول,يمكن
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
8,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# **TF-IDF**

In [11]:
tfidfvectorizer = TfidfVectorizer()

X = tfidfvectorizer.fit_transform(contentParagraphs)

df_tfidf_sklearn = pd.DataFrame(X.toarray(), columns=tfidfvectorizer.get_feature_names_out())

df_tfidf_sklearn

,2020,2024,22,38,اتصالات,اثار,اجراءات,اجرته,اخر,اذا,...,يحقق,يخوض,يد,يدعي,يرون,يزعم,يزيد,يعبرون,يقول,يمكن
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.189715,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166253,...,0.166253,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,0.000000,0.255912,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.255912,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,0.000000,0.000000,0.193531,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.193531,0.000000,0.000000,0.193531,0.000000,0.000000,0.000000,0.00000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.189911,0.00000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.194085,0.000000,0.000000,0.166682,0.194085,0.000000,0.00000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.187417,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
7,0.000000,0.000000,0.000000,0.000000,0.224293,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.224293,0.000000,0.000000,0.000000,0.000000,0.192625,0.000000,0.000000,0.00000
8,0.000000,0.000000,0.000000,0.244408,0.000000,0.000000,0.000000,0.244408,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.244408,0.000000,0.000000,0.000000,0.000000,0.00000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [12]:
for p in contentWords:
    print(p)

['وصف', 'المرشح', 'الجمهوري', 'للرءاسه', 'الاميركيه', 'دونالد', 'ترامب', 'المهاجرين', 'الشرعيين', 'الولايات', 'المتحده', 'بانهم', 'حيوانات', 'وليسوا', 'بشرا', 'خلال', 'خطاب', 'القاه', 'ولايه', 'ميشيغان', 'اثار', 'انتقادات', 'واسعه', 'لعودته', 'الي', 'اللغه', 'المهينه', 'اعتمدها', 'حملاته', 'السابقه']
['وجاءت', 'تصريحات', 'ترامب', 'خلال', 'ظهوره', 'الي', 'جانب', 'عدد', 'ضباط', 'انفاذ', 'القانون', 'تطرق', 'الي', 'عده', 'قضايا', 'جناءيه', 'تتعلق', 'بمشتبه', 'موجودين', 'الولايات', 'المتحده', 'بشكل', 'قانوني', 'وحذر', 'ان', 'العنف', 'والفوضي', 'سيدمران', 'البلاد', 'اذا', 'يحقق', 'الفوز', 'الانتخابات', 'المقرره', 'نوفمبرتشرين', 'الثاني']
['وفي', 'خطاب', 'اخر', 'ولايه', 'ويسكونسن', 'واصل', 'ترامب', 'نبره', 'التصريحات', 'المثيره', 'وصف', 'انتخابات', 'عام', '2024', 'بانها', 'المعركه', 'النهاءيه', 'للامه']
['وفيما', 'يتعلق', 'بحادثه', 'لاكين', 'رايلي', 'طالبه', 'التمريض', 'البالغه', 'العمر', '22', 'عاما', 'جورجيا', 'والتي', 'يزعم', 'انها', 'قتلت', 'علي', 'يد', 'مهاجر', 'شرعي', 'فنزويلا', 'اشار',

# **Word Embeddings**
## **Word2Vec**

In [13]:
# CBOW Model
cbow_model = Word2Vec(sentences=contentWords, vector_size=300, window=5, sg=0, min_count=1)

# Skip-gram Model
skipgram_model = Word2Vec(sentences=contentWords, vector_size=300, window=5, sg=1, min_count=1)
cbow_model_words = list(cbow_model.wv.index_to_key)
cbow_model.wv.vectors

array([[-1.7782755e-04,  9.7027631e-05,  1.7103245e-03, ...,
         1.6426874e-04,  2.7376986e-03, -2.3540137e-03],
       [-2.7441583e-03,  3.1148666e-03, -7.2869530e-05, ...,
        -2.9118538e-03,  9.8536315e-04, -2.2264996e-03],
       [ 2.7120980e-03, -1.4460933e-03, -3.4390998e-04, ...,
         2.3531297e-03,  1.9708178e-03, -1.8629067e-03],
       ...,
       [ 2.2669819e-03,  1.3487900e-03, -2.2551569e-03, ...,
        -1.5024184e-03,  6.7709375e-04, -1.3111884e-03],
       [-3.5290484e-04, -2.6651856e-03,  2.3156663e-03, ...,
        -1.6231150e-03, -2.2766599e-03,  2.6763434e-04],
       [-1.6568470e-03, -3.6584897e-04, -4.4974277e-04, ...,
         2.5481095e-03,  1.8844297e-06, -9.6234202e-04]], dtype=float32)

In [14]:
skipgram_model_words = list(skipgram_model.wv.index_to_key)
skipgram_model.wv.vectors

array([[-1.4553545e-04,  1.4188785e-04,  1.6999240e-03, ...,
         9.3936978e-05,  2.7901495e-03, -2.4097711e-03],
       [-2.7458619e-03,  3.1682670e-03, -6.2726096e-05, ...,
        -2.9508264e-03,  9.9868746e-04, -2.2481582e-03],
       [ 2.7331046e-03, -1.2171370e-03, -3.4310811e-04, ...,
         2.2841583e-03,  2.0167646e-03, -1.9477620e-03],
       ...,
       [ 2.2839003e-03,  1.3829571e-03, -2.2359130e-03, ...,
        -1.5323570e-03,  6.9452624e-04, -1.3135374e-03],
       [-3.6239953e-04, -2.5823638e-03,  2.3288918e-03, ...,
        -1.6591696e-03, -2.2563708e-03,  2.6706042e-04],
       [-1.6357624e-03, -3.4128709e-04, -4.5647484e-04, ...,
         2.5342568e-03,  3.5070876e-05, -9.8022819e-04]], dtype=float32)

In [15]:
cbow_model_X = cbow_model.wv.vectors
skipgram_model_Y = skipgram_model.wv.vectors

In [31]:
cbow_pca_3d = PCA(n_components=3)
cbow_result_3d = cbow_pca_3d.fit_transform(cbow_model_X)

skipgram_pca_3d = PCA(n_components=3)
skipgram_result_3d = skipgram_pca_3d.fit_transform(skipgram_model_Y)

In [32]:
""" 
import matplotlib.pyplot as plt

plt.style.use('_mpl-gallery')

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(result[:, 0], result[:, 1], result[:, 2])

words = list(cbow_model.wv.key_to_index.keys())
for i, word in enumerate(words):
	ax.text(result[i, 0], result[i, 1], result[i, 2], word)
	
plt.show() 
"""

" \nimport matplotlib.pyplot as plt\n\nplt.style.use('_mpl-gallery')\n\nfig = plt.figure(figsize=(10, 10))\nax = fig.add_subplot(111, projection='3d')\n\nax.scatter(result[:, 0], result[:, 1], result[:, 2])\n\nwords = list(cbow_model.wv.key_to_index.keys())\nfor i, word in enumerate(words):\n\tax.text(result[i, 0], result[i, 1], result[i, 2], word)\n\t\nplt.show() \n"

In [34]:
cbow_pca_2d = PCA(n_components=2)
cbow_result_2d = cbow_pca_2d.fit_transform(cbow_model_X)

skipgram_pca_2d = PCA(n_components=2)
skipgram_result_2d = skipgram_pca_2d.fit_transform(skipgram_model_Y)

# **GloVe**

In [35]:
""" # flatten contentWords
contentWordsFlat = [item for sublist in contentWords for item in sublist]

# Concat all words in the list
contentWordsFlat = ' '.join(contentWordsFlat) """

" # flatten contentWords\ncontentWordsFlat = [item for sublist in contentWords for item in sublist]\n\n# Concat all words in the list\ncontentWordsFlat = ' '.join(contentWordsFlat) "

In [36]:
""" # Save contentWordsFlat to txt
with open('contentWordsFlat.txt', 'w', encoding='utf-8') as f:
    f.write(contentWordsFlat) """

" # Save contentWordsFlat to txt\nwith open('contentWordsFlat.txt', 'w', encoding='utf-8') as f:\n    f.write(contentWordsFlat) "

In [ ]:
glove_input_file = 'tmp/vocab.txt'
word2vec_output_file = 'tmp/word2vec.txt'

glove2word2vec(glove_input_file, word2vec_output_file)

In [65]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(contentWords)

print("Number of unique words in dictionary=",
len(tokenizer.word_index))
print("Dictionary is = ", tokenizer.word_index)

def embedding_for_vocab(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1
    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size,embedding_dim))
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]
            return embedding_matrix_vocab
        
embedding_dim = 300
embedding_matrix_vocab = embedding_for_vocab( 'tmp/vectors.txt', tokenizer.word_index, embedding_dim)

for i in embedding_matrix_vocab:
    print(i)

Number of unique words in dictionary= 210
Dictionary is =  {'ترامب': 1, 'ان': 2, 'وفي': 3, 'الولايات': 4, 'المتحده': 5, 'الي': 6, 'ولايه': 7, 'الانتخابات': 8, 'علي': 9, 'بايدن': 10, 'المهاجرين': 11, 'بشرا': 12, 'خطاب': 13, 'ميشيغان': 14, 'انهم': 15, 'وصف': 16, 'الاميركيه': 17, 'حيوانات': 18, 'خلال': 19, 'بشكل': 20, 'قانوني': 21, 'وحذر': 22, 'العنف': 23, 'المثيره': 24, 'انتخابات': 25, 'عام': 26, 'انها': 27, 'اشار': 28, 'الحدود': 29, 'يزيد': 30, 'الرءاسيه': 31, 'الجمهوريين': 32, 'العام': 33, 'الهجره': 34, 'ويسكنسن': 35, 'بفارق': 36, 'اقل': 37, 'مءويه': 38, 'المرشح': 39, 'الجمهوري': 40, 'للرءاسه': 41, 'دونالد': 42, 'الشرعيين': 43, 'بانهم': 44, 'وليسوا': 45, 'القاه': 46, 'اثار': 47, 'انتقادات': 48, 'واسعه': 49, 'لعودته': 50, 'اللغه': 51, 'المهينه': 52, 'اعتمدها': 53, 'حملاته': 54, 'السابقه': 55, 'وجاءت': 56, 'تصريحات': 57, 'ظهوره': 58, 'جانب': 59, 'عدد': 60, 'ضباط': 61, 'انفاذ': 62, 'القانون': 63, 'تطرق': 64, 'عده': 65, 'قضايا': 66, 'جناءيه': 67, 'تتعلق': 68, 'بمشتبه': 69, 'موجودين': 70, '

In [66]:
# tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(contentWords)

word_index = tokenizer.word_index
vocab_size = len(word_index)
vocab_size

210

In [67]:
max(len(data) for data in contentWords)

36

In [68]:
# padding text data
sequences = tokenizer.texts_to_sequences(contentWords)
padded_seq = pad_sequences(sequences, maxlen=131, padding='post', truncating='post')

padded_seq[0]

array([16, 39, 40, 41, 17, 42,  1, 11, 43,  4,  5, 44, 18, 45, 12, 19, 13,
       46,  7, 14, 47, 48, 49, 50,  6, 51, 52, 53, 54, 55,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [69]:
embedding_index = {}

# https://archive.org/download/arabic_corpus
with open('tmp/vectors.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

In [70]:
embedding_index['العربية']

array([ 2.598630e-01,  1.030300e-01, -7.387420e-01, -1.000703e+00,
       -1.255200e-02, -6.178440e-01,  1.656520e-01,  5.039340e-01,
        4.468900e-02, -8.993100e-02, -3.441230e-01,  1.194600e-01,
       -2.995390e-01, -1.195000e-02,  2.095160e-01, -6.484400e-02,
       -2.726970e-01,  1.721590e-01,  1.241980e-01,  4.265000e-02,
        4.922610e-01,  6.455810e-01, -2.631620e-01, -2.451430e-01,
       -3.082910e-01, -1.804400e-01, -2.413480e-01, -1.373160e-01,
       -3.869380e-01,  1.043340e-01,  3.203740e-01, -2.627900e-01,
        1.216450e-01, -2.070500e-02,  3.428150e-01,  4.732960e-01,
       -4.634950e-01,  1.782070e-01,  2.177054e+00,  3.048700e-01,
        1.572920e-01, -7.447480e-01,  6.339500e-02, -1.953020e-01,
       -3.649560e-01,  1.045520e-01,  1.913590e-01, -3.381090e-01,
       -2.161650e-01, -2.419050e-01,  3.420180e-01, -2.716970e-01,
       -9.667700e-02, -1.445800e-01,  1.118820e-01,  3.984150e-01,
       -4.540770e-01,  2.824050e-01,  4.466370e-01, -5.628550e

In [72]:
# create embedding matrix
embedding_matrix = np.zeros((vocab_size+1, 256))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [74]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.052899  , -0.282906  , -0.013034  , ..., -0.062191  ,
         0.119889  , -0.106124  ],
       [ 0.09579   , -0.162807  ,  0.37487099, ...,  0.101266  ,
        -0.169881  ,  0.05315   ],
       ...,
       [-0.48337701, -0.070738  ,  0.027234  , ...,  0.235596  ,
         0.087646  ,  0.229899  ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.31035399,  0.149673  ,  0.820786  , ...,  0.37941501,
         0.081279  ,  0.091264  ]])

In [75]:
glove_pca_2d = PCA(n_components=2)
glove_result_2d = glove_pca_2d.fit_transform(embedding_matrix)

glove_pca_3d = PCA(n_components=3)
glove_result_3d = glove_pca_3d.fit_transform(embedding_matrix)

# **FastText**

In [37]:
# CBOW Model
fasttext_cbow_model = FastText(sentences=contentWords, vector_size=300, window=5, sg=0, min_count=1)

# Skip-gram Model
fasttext_skipgram_model = FastText(sentences=contentWords, vector_size=300, window=5, sg=1, min_count=1)

fasttext_cbow_model_words = list(fasttext_cbow_model.wv.index_to_key)
fasttext_cbow_model.wv.vectors

array([[-5.0176797e-04,  3.6458942e-04, -1.8467013e-04, ...,
         2.1500686e-04, -3.4250820e-04, -4.5722697e-04],
       [-1.3246766e-03,  1.0306223e-03, -1.1389902e-04, ...,
        -1.1813684e-03, -1.2710663e-04,  8.0339762e-04],
       [ 5.7016232e-04, -2.1480969e-03, -1.5031865e-03, ...,
         2.1796327e-03,  2.4605051e-03, -1.3805153e-03],
       ...,
       [ 3.5861987e-04, -1.3032221e-04, -4.4876474e-04, ...,
        -2.2086149e-05,  2.0741306e-04, -1.0319222e-03],
       [-8.1989398e-05,  2.0196133e-04,  9.4974886e-05, ...,
        -1.9880939e-05, -2.2570396e-04,  9.5405285e-06],
       [ 1.2039391e-04, -4.1228335e-04, -4.8440817e-04, ...,
        -3.3786704e-04, -6.1629326e-05,  8.3007399e-05]], dtype=float32)

In [38]:
fasttext_skipgram_model_words = list(fasttext_skipgram_model.wv.index_to_key)
fasttext_skipgram_model.wv.vectors

array([[-5.2714464e-04,  3.2721774e-04, -2.0824936e-04, ...,
         1.7914691e-04, -3.2120419e-04, -4.3242628e-04],
       [-1.3382516e-03,  9.9853636e-04, -1.3717760e-04, ...,
        -1.2009200e-03, -1.1260450e-04,  8.2397007e-04],
       [ 4.8586176e-04, -2.2964841e-03, -1.6286187e-03, ...,
         2.0681766e-03,  2.5115963e-03, -1.2757287e-03],
       ...,
       [ 3.3950646e-04, -1.5279840e-04, -4.7168322e-04, ...,
        -4.3268930e-05,  2.1508786e-04, -1.0134385e-03],
       [-1.4397528e-04,  9.2036644e-05, -3.6507856e-06, ...,
        -1.0804686e-04, -1.8767845e-04,  1.0568993e-04],
       [ 6.5518703e-05, -5.1949744e-04, -5.7798339e-04, ...,
        -4.1130095e-04, -2.4579300e-05,  1.5487603e-04]], dtype=float32)

In [39]:
fasttext_cbow_model_X = fasttext_cbow_model.wv.vectors
fasttext_skipgram_model_Y = fasttext_skipgram_model.wv.vectors

In [40]:
fasttext_cbow_pca_2d = PCA(n_components=2)
fasttext_cbow_result_2d = fasttext_cbow_pca_2d.fit_transform(fasttext_cbow_model_X)

fasttext_skipgram_pca_2d = PCA(n_components=2)
fasttext_skipgram_result_2d = fasttext_skipgram_pca_2d.fit_transform(fasttext_skipgram_model_Y)

In [41]:
fasttext_cbow_pca_3d = PCA(n_components=3)
fasttext_cbow_result_3d = fasttext_cbow_pca_3d.fit_transform(fasttext_cbow_model_X)

fasttext_skipgram_pca_3d = PCA(n_components=3)
fasttext_skipgram_result_3d = fasttext_skipgram_pca_3d.fit_transform(fasttext_skipgram_model_Y)

# **Visualizing Word Embeddings**

In [78]:
fig = make_subplots(rows=3, cols=2, subplot_titles=('CBOW Model', 'Skip-gram Model', 'FastText CBOW Model', 'FastText Skip-gram Model', 'GloVe Model'))

fig.add_trace(go.Scatter(x=cbow_result_2d[:, 0], y=cbow_result_2d[:, 1], mode='markers', marker=dict(size=3, color=cbow_result_2d[:, 1], colorscale='Viridis', opacity=0.8), text=cbow_model_words, hoverinfo='text'), row=1, col=1)
fig.add_trace(go.Scatter(x=skipgram_result_2d[:, 0], y=skipgram_result_2d[:, 1], mode='markers', marker=dict(size=3, color=skipgram_result_2d[:, 1], colorscale='Viridis', opacity=0.8), text=skipgram_model_words, hoverinfo='text'), row=1, col=2)
fig.add_trace(go.Scatter(x=fasttext_cbow_result_2d[:, 0], y=fasttext_cbow_result_2d[:, 1], mode='markers', marker=dict(size=3, color=fasttext_cbow_result_2d[:, 1], colorscale='Viridis', opacity=0.8), text=fasttext_cbow_model_words, hoverinfo='text'), row=2, col=1)
fig.add_trace(go.Scatter(x=fasttext_skipgram_result_2d[:, 0], y=fasttext_skipgram_result_2d[:, 1], mode='markers', marker=dict(size=3, color=fasttext_skipgram_result_2d[:, 1], colorscale='Viridis', opacity=0.8), text=fasttext_skipgram_model_words, hoverinfo='text'), row=2, col=2)
fig.add_trace(go.Scatter(x=glove_result_2d[:, 0], y=glove_result_2d[:, 1], mode='markers', marker=dict(size=3, color=glove_result_2d[:, 1], colorscale='Viridis', opacity=0.8), text=fasttext_skipgram_model_words, hoverinfo='text'), row=3, col=1)
fig.update_layout(title_text='Models', height=2400, width=1600, showlegend=False)

fig.show()

In [80]:
fig = make_subplots(rows=3, cols=2, specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}], [{'type': 'scatter3d'}, {'type': 'scatter3d'}], [{'type': 'scatter3d'}, {'type': 'scatter3d'}]], subplot_titles=('CBOW Model', 'Skip-gram Model', 'FastText CBOW Model', 'FastText Skip-gram Model', 'GloVe Model'))

fig.add_trace(go.Scatter3d(x=cbow_result_3d[:, 0], y=cbow_result_3d[:, 1], z=cbow_result_3d[:, 2], mode='markers', marker=dict(size=3, color=cbow_result_3d[:, 2], colorscale='Viridis', opacity=0.8), text=cbow_model_words, hoverinfo='text'), row=1, col=1)
fig.add_trace(go.Scatter3d(x=skipgram_result_3d[:, 0], y=skipgram_result_3d[:, 1], z=skipgram_result_3d[:, 2], mode='markers', marker=dict(size=3, color=skipgram_result_3d[:, 2], colorscale='Viridis', opacity=0.8), text=skipgram_model_words, hoverinfo='text'), row=1, col=2)
fig.add_trace(go.Scatter3d(x=fasttext_cbow_result_3d[:, 0], y=fasttext_cbow_result_3d[:, 1], z=fasttext_cbow_result_3d[:, 2], mode='markers', marker=dict(size=3, color=fasttext_cbow_result_3d[:, 2], colorscale='Viridis', opacity=0.8), text=fasttext_cbow_model_words, hoverinfo='text'), row=2, col=1)
fig.add_trace(go.Scatter3d(x=fasttext_skipgram_result_3d[:, 0], y=fasttext_skipgram_result_3d[:, 1], z=fasttext_skipgram_result_3d[:, 2], mode='markers', marker=dict(size=3, color=fasttext_skipgram_result_3d[:, 2], colorscale='Viridis', opacity=0.8), text=fasttext_skipgram_model_words, hoverinfo='text'), row=2, col=2)
fig.add_trace(go.Scatter3d(x=glove_result_3d[:, 0], y=glove_result_3d[:, 1], z=glove_result_3d[:, 2], mode='markers', marker=dict(size=3, color=glove_result_3d[:, 2], colorscale='Viridis', opacity=0.8), text=fasttext_skipgram_model_words, hoverinfo='text'), row=3, col=1)

fig.update_layout(title_text='Models', height=2400, width=1600, showlegend=False)

fig.show()

# **TSNE**

In [81]:
tsne = TSNE(n_components=2)

cbow_tsne = tsne.fit_transform(cbow_model_X)
skipgram_tsne = tsne.fit_transform(skipgram_model_Y)
fasttext_cbow_tsne = tsne.fit_transform(fasttext_cbow_model_X)
fasttext_skipgram_tsne = tsne.fit_transform(fasttext_skipgram_model_Y)
glove_tsne = tsne.fit_transform(embedding_matrix)

fig = make_subplots(rows=3, cols=2, subplot_titles=('CBOW Model', 'Skip-gram Model', 'FastText CBOW Model', 'FastText Skip-gram Model', 'GloVe Model'))

fig.add_trace(go.Scatter(x=cbow_tsne[:, 0], y=cbow_tsne[:, 1], mode='markers', marker=dict(size=3, color=cbow_tsne[:, 1], colorscale='Viridis', opacity=0.8), text=cbow_model_words, hoverinfo='text'), row=1, col=1)
fig.add_trace(go.Scatter(x=skipgram_tsne[:, 0], y=skipgram_tsne[:, 1], mode='markers', marker=dict(size=3, color=skipgram_tsne[:, 1], colorscale='Viridis', opacity=0.8), text=skipgram_model_words, hoverinfo='text'), row=1, col=2)
fig.add_trace(go.Scatter(x=fasttext_cbow_tsne[:, 0], y=fasttext_cbow_tsne[:, 1], mode='markers', marker=dict(size=3, color=fasttext_cbow_tsne[:, 1], colorscale='Viridis', opacity=0.8), text=fasttext_cbow_model_words, hoverinfo='text'), row=2, col=1)
fig.add_trace(go.Scatter(x=fasttext_skipgram_tsne[:, 0], y=fasttext_skipgram_tsne[:, 1], mode='markers', marker=dict(size=3, color=fasttext_skipgram_tsne[:, 1], colorscale='Viridis', opacity=0.8), text=fasttext_skipgram_model_words, hoverinfo='text'), row=2, col=2)
fig.add_trace(go.Scatter(x=glove_tsne[:, 0], y=glove_tsne[:, 1], mode='markers', marker=dict(size=3, color=glove_tsne[:, 1], colorscale='Viridis', opacity=0.8), text=fasttext_skipgram_model_words, hoverinfo='text'), row=3, col=1)

fig.update_layout(title_text='Word2Vec Models', height=2400, width=1600, showlegend=False)

fig.show()

# **REFERENCES**
[1] Motazsaad, “GitHub - motazsaad/process-arabic-text: Pre-process arabic text (remove diacritics, punctuations and repeating characters),” GitHub. https://github.com/motazsaad/process-arabic-text/

[2] T. Zerrouki, “pyarabic, an arabic language library for python.”, 2010. [Online]. Available: https://pypi.python.org/pypi/pyarabic/

[3] ——, “Tashaphyne, arabic light stemmer,” 2012. [Online]. Available: https://pypi.python.org/pypi/Tashaphyne/

[4] ——, “qalsadi, arabic mophological analyzer library for python.” 2012. [Online]. Available: https://pypi.python.org/pypi/qalsadi/

[5] ——, “Towards an open platform for arabic language processing,” 2020.

[6] A. L. T. G. at Qatar Computing Research Institute (QCRI), “farasa, the state-of-the-art full-stack package to deal with arabic language processing.” 2020. [Online]. Available: https://farasa.qcri.org/

[7] MagedSaeed, “GitHub - MagedSaeed/farasapy: A Python implementation of Farasa toolkit,” GitHub. https://github.com/MagedSaeed/farasapy/

[8] M. H. Btoush, A. Alarabeyyat, and I. Olab, “Rule based approach for arabic part of speech tagging and name entity recognition,” International Journal of Advanced Computer Science and Applications, vol. 7, no. 6, 2016. [Online]. Available: http://dx.doi.org/10.14569/IJACSA.2016.070642/